In [30]:
import os
os.chdir('/workspace/FutureGPT2/src/')
os.environ['HF_HOME'] = '/workspace/cache/huggingface/'

import numpy as np
from torch import optim, nn, Tensor
from torch.nn import functional as F
import torch
import wandb
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
import transformers
import lightning as L
from inspect import signature, _ParameterKind
import copy
import gc
import datasets
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
from datasets import load_dataset
from itertools import islice

from lightning.pytorch.callbacks import ModelCheckpoint, LearningRateMonitor
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.loggers import WandbLogger

In [2]:
#train = datasets.load_from_disk('/workspace/corpus/msmarco/msmarco_GPT2_64tokens_full/train').with_format('torch')
val = datasets.load_from_disk('/workspace/corpus/msmarco/msmarco_GPT2_64tokens_full/val').with_format('torch', device='cuda')

In [3]:
#train_loader = DataLoader(train, batch_size=128, num_workers=50)
val_loader = DataLoader(val, batch_size=128)

In [67]:
gpt_tokenizer = AutoTokenizer.from_pretrained('gpt2')
GPTToken = {v: k for k, v in gpt_tokenizer.get_vocab().items()}

In [20]:
gpt = AutoModelForCausalLM.from_pretrained('gpt2').to('cuda')

In [49]:
pythia = AutoModelForCausalLM.from_pretrained('EleutherAI/pythia-410m-deduped').to('cuda')

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/911M [00:00<?, ?B/s]

In [68]:
pythia_tokenizer = AutoTokenizer.from_pretrained('EleutherAI/pythia-410m-deduped')
PythiaToken = {v: k for k, v in pythia_tokenizer.get_vocab().items()}

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [21]:
torch.set_grad_enabled(False)

In [6]:
total = 0
loss = 0
for batch in tqdm(iter(val_loader)):
    del batch['text']
    del batch['id']
    out = gpt(**batch, labels=batch['input_ids'])
    loss += out.loss.item()
    total += 1

  0%|          | 0/405 [00:00<?, ?it/s]

In [8]:
loss / total

3.691645735870173

In [22]:
state = torch.load('/workspace/checkpoints/GPT2-MSMARCO-COSINE_global_step=9099.0_val_loss=3.28.ckpt')['state_dict']
own = AutoModelForCausalLM.from_pretrained('gpt2').to('cuda')
own.load_state_dict({'.'.join(k.split('.')[1:]): v for k, v in state.items()})

<All keys matched successfully>

In [23]:
state = torch.load('/workspace/checkpoints/GPT2-MYOPIC-CUTGRAD_val_myopic_loss=3.73.ckpt')['state_dict']
myopic = AutoModelForCausalLM.from_pretrained('gpt2').to('cuda')
myopic.load_state_dict({'.'.join(k.split('.')[1:]): v for k, v in state.items()})

<All keys matched successfully>

In [20]:
total = 0
loss = 0
for batch in tqdm(iter(val_loader)):
    del batch['text']
    del batch['id']
    out = own(**batch, labels=batch['input_ids'])
    loss += out.loss.item()
    total += 1

  0%|          | 0/405 [00:00<?, ?it/s]

In [21]:
loss / total

3.2765459608148646

In [74]:
evals = [
    'It was a dark and stormy',
    'Anna was born in Hungary, so her first language is',
    #'The store sells only red pens and blue books, so I bought a',
    #'The color of the sky is',
    #'One plus one is',
    'The director of the Manhattan project was J. Robert Opp',
    'These are my two friends Tom Smith and Bob Jones. Of the two, my favorite is',
    #'It was a dark and stormy',
]

In [75]:
for s in evals:
    for model, name in zip([gpt, own, myopic, pythia], ['THEIRS', 'MSMARCO', 'MYOPIC', 'PYTHIA']):
        tokenizer = pythia_tokenizer if name == 'PYTHIA' else gpt_tokenizer
        Token = PythiaToken if name == 'PYTHIA' else GPTToken
        input = tokenizer(
            s, 
            return_tensors='pt'
        ).to('cuda')
        out = model.generate(
            **input,
            max_new_tokens=40,
            pad_token_id=tokenizer.eos_token_id,
            temperature=0,
        )
        print(name)
        print(out.shape[1], 'tokens')
        out = ''.join([Token[x].replace('Ġ', ' ').replace('Ċ', ' ') for x in list(out.flatten().cpu().numpy())])
        out = out.replace(s, s + '||')
        print(out)
    print()

THEIRS
47 tokens
It was a dark and stormy|| night. The wind was blowing, and the clouds were falling. The wind was blowing, and the clouds were falling. The wind was blowing, and the clouds were falling. The wind was blowing,
MSMARCO
47 tokens
It was a dark and stormy|| day, and the sun was shining. The sun was shining, and the sun was shining. The sun was shining, and the sun was shining. The sun was shining, and the sun was shining
MYOPIC
47 tokens
It was a dark and stormy|| day in the middle of the night. The first day of the month was the first day of the month, and the second day of the month. The first day of the month is the day of
PYTHIA
47 tokens
It was a dark and stormy|| night, and the wind was howling.  "I'm sorry," he said. "I'm sorry."  "I'm sorry," she said. "I'm sorry." 

THEIRS
51 tokens
Anna was born in Hungary, so her first language is|| Hungarian. She is a very good student, and she is very good at math. She is very good at reading. She is very good at reading. She 

In [69]:
out = own.generate(
    **input,
    max_new_tokens=30,
)
''.join([Token[x].replace('Ġ', ' ') for x in list(out.flatten().cpu().numpy())])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Alice lives in France, so she speaks to her. She is a very good-looking woman, and she is a very good-looking woman. She is a very good-looking woman'

In [70]:
out = myopic.generate(
    **input,
    max_new_tokens=30,
)
''.join([Token[x].replace('Ġ', ' ') for x in list(out.flatten().cpu().numpy())])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'Alice lives in France, so she speaks to the world of the world. He is the son of a man who is the son of a man who is the son of a man. He'

In [3]:
owt = load_dataset('openwebtext', split='train[:100000]')

/home/wwu/.local/lib/python3.10/site-packages/datasets/load.py:1429: FutureWarning: The repository for openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/openwebtext
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/8013769 [00:00<?, ? examples/s]

In [76]:
for model, name in [(pythia, 'PYTHIA')]:#zip([gpt, own, myopic], ['THEIRS', 'OURS', 'MYOPIC']):
    total = 0
    loss = 0
    tokenizer = pythia_tokenizer if name == 'PYTHIA' else gpt_tokenizer
    for batch in tqdm(islice(iter(owt_loader), 100), total=100):
        input = tokenizer(batch['text'], return_tensors='pt', truncation=True, max_length=64).to('cuda')
        out = model(**input, labels=input['input_ids'])
        loss += out.loss.item()
        total += 1
    print(name, loss / total)

  0%|          | 0/100 [00:00<?, ?it/s]

PYTHIA 3.1627233052253723


In [ ]:
total = 0
loss = 0
for batch in tqdm(iter(owt_loader)):
    input = tokenizer(batch['text'], return_tensors='pt', truncation=True, max_length=64).to('cuda')
    out = gpt(**input, labels=input['input_ids'])
    loss += out.loss.item()
    total += 1
loss / total